# Statistical Hypothesis Testing

In [31]:
import math
import random

In [2]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomail(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [3]:
def normal_cdf(x, mu=0, sigma=1):
    return(1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

In [16]:
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    
    # if not standard, compute standard and rescale
    if mu != 0 or sigma !=1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0 #normal_cdf(-10) is very close to 0
    hi_z = 10.0 #normal_csf(10) is very close to 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2 # consider the midpoint
        mid_p = normal_cdf(mid_z)
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z = mid_z
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z = mid_z
        else:
            break
    return mid_z

In [6]:
# the normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = normal_cdf

In [4]:
# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

In [5]:
#it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

In [7]:
# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [8]:
def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

In [9]:
def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

In [13]:
def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """return the symmetric (about the mean) bounds
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [14]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [17]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

In [18]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [19]:
# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

In [20]:
# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in. our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

In [21]:
power

0.8865480012953671

In [22]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)

In [23]:
hi

526.0073585242053

In [24]:
# 526 < 531 since we need more priobability in the upper tail

In [25]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability

In [26]:
power

0.9363794803307173

# p-values

In [28]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)

In [29]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

## we use 529.5 instead of 330 because of whats called a __[connunity correction](https://en.wikipedia.org/wiki/Continuity_correction)__

In [33]:
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                   for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1
print(extreme_value_count/100000)

0.06139


In [34]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [35]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [36]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

In [37]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

# Confidence Intervals

In [38]:
p_hat = 525 / 1000

In [39]:
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

In [40]:
print(mu, sigma)

0.525 0.015791611697353755


In [41]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

In [42]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

# P-hacking

In [43]:
def run_experiment():
    """flip a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

In [47]:
def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [48]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                     for experiment in experiments
                     if reject_fairness(experiment)])
print(num_rejections)

46


# Running an A/B Test

In [53]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [54]:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_A - p_B) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [55]:
z = a_b_test_statistic(1000, 200, 1000, 180)
print(z)

1.1403464899034472


In [57]:
two_sided_p_value(z)

0.25414197654223614

In [58]:
z = a_b_test_statistic(1000, 200, 1000, 150)
print(z)

2.948839123097944


In [59]:
two_sided_p_value(z)

0.003189699706216853

# Bayesian Inference

In [60]:
def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

In [61]:
def beta_pdf(x, alpha, beta):
    if x <= 0 or x >= 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)